# RFM Customer Segmentation Flask Project

This notebook contains:

- Instructions to run the Flask app
- The Flask `app.py` code
- HTML templates (`index.html`, `rfm.html`)
- A sample CSV and a demonstration cell to compute RFM in the notebook

You can download the notebook from the link at the end of the notebook after it is generated.

## 1. app.py

The Flask application that serves the web interface, processes uploaded CSVs, computes RFM, and provides CSV download.

In [ ]:
# Save the Flask app code to a file named app.py
app_py = r"""
import os
from flask import Flask, render_template, send_file, request, redirect, url_for
import pandas as pd
from datetime import datetime
import io

app = Flask(__name__, template_folder="templates")

# Global uploaded dataframe
uploaded_df = None

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/upload_csv", methods=["POST"])
def upload_csv():
    global uploaded_df
    if 'file' not in request.files:
        return "No file part", 400
    file = request.files['file']
    if file.filename == '':
        return "No selected file", 400
    try:
        uploaded_df = pd.read_csv(file)
    except Exception as e:
        return f"Error reading CSV: {e}", 400
    return redirect(url_for('rfm_output'))

@app.route("/rfm")
def rfm_output():
    global uploaded_df

    # Use uploaded CSV if available, else sample data
    if uploaded_df is not None:
        df = uploaded_df.copy()
    else:
        data = {
            'CustomerID': [1, 1, 2, 2, 2, 3, 4, 4, 5],
            'InvoiceDate': [
                '2023-01-01', '2023-02-15', '2023-01-10',
                '2023-03-20', '2023-04-05', '2023-02-01',
                '2023-01-15', '2023-03-10', '2023-05-01'
            ],
            'Amount': [100, 150, 200, 50, 300, 75, 120, 80, 250]
        }
        df = pd.DataFrame(data)

    # Ensure InvoiceDate is datetime
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

    reference_date = datetime(2023, 6, 1)

    rfm = df.groupby('CustomerID').agg(
        Recency=('InvoiceDate', lambda x: (reference_date - x.max()).days),
        Frequency=('InvoiceDate', 'count'),
        Monetary=('Amount', 'sum')
    ).reset_index()

    def safe_qcut(series, bins, labels):
        try:
            return pd.qcut(series.rank(method='first'), bins, labels=labels).astype(int)
        except Exception:
            return pd.Series([3] * len(series))

    rfm['R_Score'] = safe_qcut(rfm['Recency'], 5, [5,4,3,2,1])
    rfm['F_Score'] = safe_qcut(rfm['Frequency'], 5, [1,2,3,4,5])
    rfm['M_Score'] = safe_qcut(rfm['Monetary'], 5, [1,2,3,4,5])

    rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

    def segment_customer(row):
        r, f, m = row['R_Score'], row['F_Score'], row['M_Score']
        if r >= 4 and f >= 4 and m >= 4:
            return 'Champions'
        elif r >= 3 and f >= 3 and m >= 3:
            return 'Loyal Customers'
        elif r >= 3 and f <= 2:
            return 'New Customers'
        elif r <= 2 and f >= 3:
            return 'At Risk'
        elif r <= 2 and f <= 2:
            return 'Lost'
        else:
            return 'Others'

    rfm['Segment'] = rfm.apply(segment_customer, axis=1)
    rfm_html = rfm.to_html(classes="table table-bordered", index=False)

    return render_template("rfm.html", table=rfm_html)

@app.route("/download_csv")
def download_csv():
    # Use the same uploaded_df or sample as in /rfm
    global uploaded_df
    if uploaded_df is not None:
        df = uploaded_df.copy()
    else:
        data = {
            'CustomerID': [1, 1, 2, 2, 2, 3, 4, 4, 5],
            'InvoiceDate': [
                '2023-01-01', '2023-02-15', '2023-01-10',
                '2023-03-20', '2023-04-05', '2023-02-01',
                '2023-01-15', '2023-03-10', '2023-05-01'
            ],
            'Amount': [100, 150, 200, 50, 300, 75, 120, 80, 250]
        }
        df = pd.DataFrame(data)

    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
    reference_date = datetime(2023, 6, 1)
    rfm = df.groupby('CustomerID').agg(
        Recency=('InvoiceDate', lambda x: (reference_date - x.max()).days),
        Frequency=('InvoiceDate', 'count'),
        Monetary=('Amount', 'sum')
    ).reset_index()

    rfm['R_Score'] = pd.qcut(rfm['Recency'].rank(method='first'), 5, labels=[5,4,3,2,1]).astype(int)
    rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)
    rfm['M_Score'] = pd.qcut(rfm['Monetary'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)
    rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

    def segment_customer(row):
        r, f, m = row['R_Score'], row['F_Score'], row['M_Score']
        if r >= 4 and f >= 4 and m >= 4:
            return 'Champions'
        elif r >= 3 and f >= 3 and m >= 3:
            return 'Loyal Customers'
        elif r >= 3 and f <= 2:
            return 'New Customers'
        elif r <= 2 and f >= 3:
            return 'At Risk'
        elif r <= 2 and f <= 2:
            return 'Lost'
        else:
            return 'Others'

    rfm['Segment'] = rfm.apply(segment_customer, axis=1)

    buffer = io.StringIO()
    rfm.to_csv(buffer, index=False)
    buffer.seek(0)
    return send_file(
        io.BytesIO(buffer.getvalue().encode()),
        mimetype="text/csv",
        as_attachment=True,
        download_name="rfm_output.csv"
    )

if __name__ == "__main__":
    app.run(debug=True)
"""
with open("app.py","w",encoding="utf-8") as f:
    f.write(app_py)
print("Saved app.py")

## 2. Templates

Save the following HTML files into a `templates/` folder next to `app.py`. The notebook will create them for you in the next cell.

In [ ]:
# Create templates directory and save index.html and rfm.html
os.makedirs("templates", exist_ok=True)

index_html = r"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Customer Segmentation Project</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.2/dist/css/bootstrap.min.css">
</head>
<body class="p-4">
    <div class="container text-center mt-5">
        <h1 class="mb-4">Customer Segmentation Using RFM</h1>
        <p class="mb-4">Analyze your customer data and segment customers based on Recency, Frequency, and Monetary values.</p>
        <!-- File Upload Form -->
        <form action="/upload_csv" method="POST" enctype="multipart/form-data" class="mb-3">
            <div class="input-group justify-content-center">
                <input type="file" name="file" class="form-control w-50" required>
                <button class="btn btn-success" type="submit">Upload CSV</button>
            </div>
        </form>
        <a href="/rfm" class="btn btn-primary btn-lg">View RFM Analysis</a>
    </div>
</body>
</html>
"""

rfm_html = r"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>RFM Analysis</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.2/dist/css/bootstrap.min.css">
</head>
<body class="p-4">
    <div class="container">
        <h2 class="mb-4">RFM Analysis</h2>
        <div class="table-responsive">
            {{ table | safe }}
        </div>
        <div class="mt-4">
            <a href="/download_csv" class="btn btn-success">Download CSV</a>
        </div>
        <div class="mt-2">
            <a href="/" class="btn btn-secondary">Back to Home</a>
        </div>
    </div>
</body>
</html>
"""

with open("templates/index.html","w",encoding="utf-8") as f:
    f.write(index_html)
with open("templates/rfm.html","w",encoding="utf-8") as f:
    f.write(rfm_html)
print("Saved templates/index.html and templates/rfm.html")

## 3. Sample CSV and RFM demo

This cell creates a sample CSV and demonstrates computing RFM directly in the notebook.

In [ ]:
# Create a sample CSV file
import pandas as pd
sample_data = {
    'CustomerID': [1,1,2,2,3,3,3,4,5],
    'InvoiceDate': ['2023-01-01','2023-02-15','2023-01-10','2023-03-20','2023-02-01','2023-04-01','2023-05-01','2023-03-10','2023-05-01'],
    'Amount': [100,150,200,50,75,60,90,80,250]
}
sample_df = pd.DataFrame(sample_data)
sample_df.to_csv("sample_customers.csv", index=False)
print("Saved sample_customers.csv")
sample_df.head()

In [ ]:
# Demonstrate RFM calculation in-notebook
df = pd.read_csv("sample_customers.csv")
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
from datetime import datetime
reference_date = datetime(2023,6,1)

rfm = df.groupby('CustomerID').agg(
    Recency=('InvoiceDate', lambda x: (reference_date - x.max()).days),
    Frequency=('InvoiceDate', 'count'),
    Monetary=('Amount', 'sum')
).reset_index()

rfm

## 4. Requirements and README

This cell will write `requirements.txt` and `README.md` files into the project folder.

In [ ]:
# Write requirements.txt and README.md
req = """Flask
pandas
numpy
python-dateutil
"""
with open("requirements.txt","w",encoding="utf-8") as f:
    f.write(req)

readme = r"""# Customer Segmentation Using RFM

## Overview
This project performs **customer segmentation** using the **RFM (Recency, Frequency, Monetary) model**. It allows users to upload their own CSV files containing customer purchase data and outputs a table showing customer segments and RFM scores. Users can also download the RFM data as a CSV.

## Usage

1. Install dependencies:

```bash
pip install -r requirements.txt
```

2. Run the Flask app:

```bash
python app.py
```

3. Open the app in your browser at http://127.0.0.1:5000/
"""

with open("README.md","w",encoding="utf-8") as f:
    f.write(readme)

print("Saved requirements.txt and README.md")

## 5. Download the notebook

The notebook file has been saved to `/mnt/data/rfm_project.ipynb`. Use the link below to download it.

In [ ]:
nb_path = "/mnt/data/rfm_project.ipynb"
import nbformat
with open(nb_path, "w", encoding="utf-8") as f:
    nbformat.write(nb, f)
print("Notebook saved to", nb_path)